# OpenAI Assistant with Autogen and Postgres

This example hows how you can use the OpenAI Assistant agents with autogen, to solve database related tasks (Postgres DB).<br>
This code takes a lot of inspiration from IndyDevDan's videos.


One thing i noticed so far: its fast! it doesn't waste time talking back and foward with the `UserProxyAgent` before writing the query - potentially saving money on token costs, i'll need to look into this further. It also seems counter intuitive because usually the `UserProxyAgent` is the one who is executing the code. it's a great addition to autogen from what i can tell so far.

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

from modules import llm
from modules import rand
from turbo4 import Turbo4
from modules import embeddings
from modules.instruments import PostgresAgentInstruments

import autogen
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent


DB_URL = os.environ.get("POSTGRES_CONNECTION_URL")
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"

In [ ]:
# ! pip install --upgrade pyautogen
# https://github.com/microsoft/autogen/releases/tag/0.2.0b5

In [2]:
# Note: this is formatted differently for autogen than if 
# you were to use the Assistants API directly.
run_sql_tool_config = {
    "name": "run_sql",
    "description": "This is a function that allows you to run SQL queries on a Postgres DB by passing the SQL as a string.",
    "parameters": {
        "type": "object",
        "properties": {
            "sql": {
                "type": "string",
                "description": "This is a string containing SQL for querying a postgres database."
            }
        },
        "required" : [
            "sql"
        ]
    },
}

# Configuration for GPTAssistantAgent
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../../.env',
    model_api_key_map={
        "gpt-4-1106-preview": "OPENAI_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
        }
    }
)

# Initialize User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config={
        "work_dir": "./autogen_results",
        "use_docker": False
    },
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

In [3]:
SQL_ASSISTANT_SYSTEM_MESSAGE = """
You're an elite SQL developer. You generate the most concise and performant SQL queries. Use the  provided tool `run_sql` to run the SQL you've just generated."
Say TERMINATE when everything is done and the overall task is complete.
"""

In [4]:
raw_prompt = "Get the mean age of those who have had covid."
prompt = f"Fulfill this database query: {raw_prompt}. "

assistant_name = "test"
session_id = rand.generate_session_id(assistant_name + raw_prompt)

with PostgresAgentInstruments(DB_URL, session_id) as (agent_instruments, db):
    
    llm_config = {
        "config_list": config_list,
        "assistant_id": None,
        "tools": [
            {
                "type": "function",
                "function": run_sql_tool_config,
            }
        ],
        "model": "gpt-4-1106-preview"
    }

    # Initialize the assistant
    sql_assistant = GPTAssistantAgent(
        name="SQL_Assistant",
        instructions=SQL_ASSISTANT_SYSTEM_MESSAGE,
        llm_config=llm_config
    )

    # Register the run_sql function
    sql_assistant.register_function(
        function_map={
            "run_sql": db.run_sql
        }
    )

    # Get table definitions from Postgres
    database_embedder = embeddings.DatabaseEmbedder(db)
    table_definitions = database_embedder.get_similar_table_defs_for_prompt(
        raw_prompt
    )

    # Forumalte prompt
    prompt = llm.add_cap_ref(
        prompt,
        f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query.",
        POSTGRES_TABLE_DEFINITIONS_CAP_REF,
        table_definitions,
    )

    # Initiate autogen chat
    user_proxy.initiate_chat(sql_assistant, message=prompt)

get_or_create_assistant(test, gpt-4-1106-preview)


user_proxy (to SQL_Assistant):

Fulfill this database query: Get the mean age of those who have had covid..  Use these TABLE_DEFINITIONS to satisfy the database query.

TABLE_DEFINITIONS

CREATE TABLE patients (
id integer,
patient_id character varying(255),
state character varying(2),
fips integer,
diagnosed_covid boolean,
diagnoses_date date,
current_age integer,
age_at_diagnosis double precision
);

CREATE TABLE patient_medical_history (
id integer,
patient_id character varying(255),
has_diabetes boolean,
has_heart_disease boolean,
has_lung_disease boolean,
smoker_status character varying(10),
bmi double precision,
last_checkup_date date
);

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION run_sql...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(run_sql, Sucess: True) : [
    {
        "avg": "47.9756097560975610"
    }
]


SQL_Assistant (to user_proxy):

The mean age of patients who have had COVID is approximately 47.98 years. 

TERMINATE


--------------------------------------------------------------------------------
